In [1]:
import numpy as np
import pandas as pd
import sys
from sklearn.metrics.pairwise import manhattan_distances

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indic

In [2]:
f=open('/Users/iamlira/Downloads/IOTest2.txt')

In [3]:
for tmp in f.readlines():
    print(tmp)

AddPou  2018/9/4 21:46:19

choose_start  2018/9/4 21:46:45

choose_end  2018/9/4 21:46:46

choose_start  2018/9/4 21:47:05

choose_end  2018/9/4 21:47:07

choose_start  2018/9/4 21:47:11

choose_end  2018/9/4 21:47:12

choose_start  2018/9/4 21:47:16

choose_end  2018/9/4 21:47:17

choose_start  2018/9/4 21:47:23

choose_end  2018/9/4 21:47:24

choose_start  2018/9/4 21:47:28

choose_end  2018/9/4 21:47:29

choose_start  2018/9/4 21:47:35

choose_end  2018/9/4 21:47:36

choose_start  2018/9/4 21:47:43

choose_end  2018/9/4 21:47:44

choose_start  2018/9/4 21:47:52

choose_end  2018/9/4 21:47:54

move_start  2018/9/4 21:48:22

connect  2018/9/4 21:48:22

move_end  2018/9/4 21:48:22

move_start  2018/9/4 21:48:23

connect  2018/9/4 21:48:23

move_end  2018/9/4 21:48:23

move_start  2018/9/4 21:48:24

connect  2018/9/4 21:48:24

move_end  2018/9/4 21:48:24

move_start  2018/9/4 21:48:25

connect  2018/9/4 21:48:26

move_end  2018/9/4 21:48:26

move_start  2018/9/4 21:48:27

connect  2018/

In [1]:
def lcs(a,b):
  lena=len(a)
  lenb=len(b)
  c=[[0 for i in range(lenb+1)] for j in range(lena+1)]
  flag=[[0 for i in range(lenb+1)] for j in range(lena+1)]
  for i in range(lena):
    for j in range(lenb):
      if a[i]==b[j]:
        c[i+1][j+1]=c[i][j]+1
        flag[i+1][j+1]='ok'
      elif c[i+1][j]>c[i][j+1]:
        c[i+1][j+1]=c[i+1][j]
        flag[i+1][j+1]='left'
      else:
        c[i+1][j+1]=c[i][j+1]
        flag[i+1][j+1]='up'
  return c,flag

def printLcs(flag,a,i,j):
  if i==0 or j==0:
    return
  if flag[i][j]=='ok':
    printLcs(flag,a,i-1,j-1)
    print(a[i-1],end='')
  elif flag[i][j]=='left':
    printLcs(flag,a,i,j-1)
  else:
    printLcs(flag,a,i-1,j)
    
a='ABCBDAB'
b='BDCABA'
c,flag=lcs(a,b)
for i in c:
  print(i)
print('')
for j in flag:
  print(j)
print('')
printLcs(flag,a,len(a),len(b))
print('')

[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1]
[0, 1, 1, 1, 1, 2, 2]
[0, 1, 1, 2, 2, 2, 2]
[0, 1, 1, 2, 2, 3, 3]
[0, 1, 2, 2, 2, 3, 3]
[0, 1, 2, 2, 3, 3, 4]
[0, 1, 2, 2, 3, 4, 4]

[0, 0, 0, 0, 0, 0, 0]
[0, 'up', 'up', 'up', 'ok', 'left', 'ok']
[0, 'ok', 'left', 'left', 'up', 'ok', 'left']
[0, 'up', 'up', 'ok', 'left', 'up', 'up']
[0, 'ok', 'up', 'up', 'up', 'ok', 'left']
[0, 'up', 'ok', 'up', 'up', 'up', 'up']
[0, 'up', 'up', 'up', 'ok', 'up', 'ok']
[0, 'ok', 'up', 'up', 'up', 'ok', 'up']

BCBA


In [12]:
def dtw(X,Y):
     M=[[manhattan_distances(X[i],Y[i]) for i in range(len(X))] for j in range(len(Y))]
     l1=len(X)
     l2=len(Y) 
     D=[[0 for i in range(l1+1)] for i in range(l2+1)]
     D[0][0]=0 
     for i in range(1,l1+1):
         D[0][i]=sys.maxsize
     for j in range(1,l2+1):
         D[j][0]=sys.maxsize
     for j in range(1,l2+1):
         for i in range(1,l1+1):
             D[j][i]=M[j-1][i-1]+min(D[j-1][i],D[j][i-1],D[j-1][i-1]+M[j-1][i-1])
     print(D[-1][-1])

In [13]:
X=[1,2,3,4]
Y=[1,2,7,4,5]
dtw(X,Y)

[[4.]]
